# Stock Anaylsis Project
## Dan Luoma

A Python script for analyzing the previous 10 days performance of selected range of stocks and ranking them using OBV analysis. <br> With the predictions of this script I intend to invest in the top three performing stocks according to their OBV ranking.

## Imports

yfinance uses Yahoo Finance API and imports stocks to be analyzed. <br>get-all-tickers uses yfinance to filter the available stock information to obtain the stocks we are interested in analyzing.

In [ ]:
!pip3 install yfinance
!pip install get-all-tickers
import yfinance as yf, pandas as pd, shutil, os, time, glob, smtplib, ssl
from get_all_tickers import get_tickers as gt

In [ ]:
# List of the stocks we are interested in analyzing with a minimum market cap specified
tickers = gt.get_tickers_filtered(mktcap_min=200)
# Print the number of stocks imported to tickers
print("The amount of stocks chosen to observe: " + str(len(tickers)))

##  Stock File Creation

Deletes the previous days stock information, recreates the folder, and begins creating csv files for all stocks being analyzed. <br>The number of stocks being analyzed has been limited to 1800 to avoid running into import issues with yfinance. <br>The path C:/Users/dluom will need to be changed to correspond to your own file system.

In [ ]:
# These two lines remove the Stocks folder and then recreate it in order to remove old stocks.
shutil.rmtree("C:/Users/dluom/Daily_Stock_Report/Stocks/")
os.mkdir("C:/Users/dluom/Daily_Stock_Report/Stocks/")
# Holds the amount of API calls we executed
Amount_of_API_Calls = 0
# This while loop is reponsible for storing the historical data for each ticker in our list.
Stock_Failure = 0  # Used to make sure we don't waste too many API calls on one Stock ticker that could be having issues
Stocks_Not_Imported = 0
# Used to iterate through our list of tickers
i=0
while (i < len(tickers)) and (Amount_of_API_Calls < 1800):
    try:
        stock = tickers[i]  # Gets the current stock ticker
        temp = yf.Ticker(str(stock))
        Hist_data = temp.history(period="max")  # Tells yfinance what kind of data we want about this stock (In this example, all of the historical data)
        Hist_data.to_csv("C:/Users/dluom/Daily_Stock_Report/Stocks/"+stock+".csv")  # Saves the historical data in csv format for further processing later
        time.sleep(2)  # Pauses the loop for two seconds so we don't cause issues with Yahoo Finance's backend operations
        Amount_of_API_Calls += 1 
        Stock_Failure = 0
        i += 1  # Iteration to the next ticker
    except ValueError:
        print("Yahoo Finance Backend Error, Attempting to Fix")  # An error occured on Yahoo Finance's backend. We will attempt to retreive the data again
        if Stock_Failure > 5:  # Move on to the next ticker if the current ticker fails more than 5 times
            i+=1
            Stocks_Not_Imported += 1
        Amount_of_API_Calls += 1
        Stock_Failure += 1
print("The amount of stocks we successfully imported: " + str(i - Stocks_Not_Imported))

## OBV Analysis

Performs the OBV analysis for each stock selected. Obtains an OBV score, places them in a seperate csv file, and orders them from greatest to least. This csv file is then saved to the file created in the previous section.<br>OBV analyzis if a stock price has increased or decreased from one day to the next, compares the volume of the stock traded, and assigns it a value. For more information please see <a>href="https://www.investopedia.com/terms/o/onbalancevolume.asp">this link.</a>

In [ ]:
list_files = (glob.glob("C:/Users/dluom/Daily_Stock_Report/Stocks/*.csv")) # Creates a list of all csv filenames in the stocks folder
new_data = [] #  This will be a 2D array to hold our stock name and OBV score
interval = 0  # Used for iteration
while interval < len(list_files):
    print(interval)
    Data = pd.read_csv(list_files[interval]).tail(10)  # Gets the last 10 days of trading for the current stock in iteration
    pos_move = []  # List of days that the stock price increased
    neg_move = []  # List of days that the stock price increased
    OBV_Value = 0  # Sets the initial OBV_Value to zero
    count = 0
    if(len(Data) < 10):
        interval += 1
        continue
    while (count < 10):  # 10 because we are looking at the last 10 trading days
        if Data.iloc[count,1] < Data.iloc[count,4]:  # True if the stock increased in price
            pos_move.append(count)  # Add the day to the pos_move list
        elif Data.iloc[count,1] > Data.iloc[count,4]:  # True if the stock decreased in price
            neg_move.append(count)  # Add the day to the neg_move list
        count += 1
    count2 = 0
    for i in pos_move:  # Adds the volumes of positive days to OBV_Value, divide by opening price to normalize across all stocks
        OBV_Value = round(OBV_Value + (Data.iloc[i,5]/Data.iloc[i,1]))
    for i in neg_move:  # Subtracts the volumes of negative days from OBV_Value, divide by opening price to normalize across all stocks
        OBV_Value = round(OBV_Value - (Data.iloc[i,5]/Data.iloc[i,1]))
    Stock_Name = ((os.path.basename(list_files[interval])).split(".csv")[0])  # Get the name of the current stock we are analyzing
    new_data.append([Stock_Name, OBV_Value])  # Add the stock name and OBV value to the new_data list
    interval += 1
df = pd.DataFrame(new_data, columns = ['Stock', 'OBV_Value'])  # Creates a new dataframe from the new_data list
df["Stocks_Ranked"] = df["OBV_Value"].rank(ascending = False)  # Rank the stocks by their OBV_Values
df.sort_values("OBV_Value", inplace = True, ascending = False)  # Sort the ranked stocks
df.to_csv("C:/Users/dluom/Daily_Stock_Report/Stocks/OBV_Ranked.csv", index = False)  # Save the dataframe to a csv without the index column